HRC Case with instantaneus action.

In [2]:
!pip install --pre unified-planning[fmap]
# !apt install libgraphviz-dev
# !pip install pygraphviz


Scheduling example:

In [3]:
from unified_planning.model.scheduling import SchedulingProblem
from unified_planning.shortcuts import *

problem = SchedulingProblem("factory")
workers = problem.add_resource("workers", capacity=4)
machine1 = problem.add_resource("machine1", capacity=1)
machine2 = problem.add_resource("machine2", capacity=1)

a1 = problem.add_activity("a1", duration=3)
a1.uses(machine1)
a1.uses(workers, amount=2)
a1.add_deadline(14)

a2 = problem.add_activity("a2", duration=6)
a2.uses(workers)
a2.uses(machine2)

problem.add_constraint(LT(a2.end, a1.start))

# One worker is unavailable over [16, 25)
problem.add_decrease_effect(16, workers, 1)
problem.add_increase_effect(25, workers, 1)

with OneshotPlanner(name='fmap') as planner:
    result = planner.solve(problem, None, "1")
    if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
        print("%s Returned Sequential Plans object: %s" % (planner.name, result.plan.all_sequential_plans()))
        # [print(f"{idx} Sequential Plans: {seq_plan}") for idx, seq_plan in enumerate(result.plan.all_sequential_plans())]
        # print("Adjacency list:", result.plan.get_adjacency_list)
        print("result:", result)
    else:
        print("Log Error:", result)